<h2 align="center">AtliQ Hotels Data Analysis Project<h2>

In [1]:
import pandas as pd

***
### ==> 1. Data Import and Data Exploration
***

### Datasets
We have 5 csv file 

   - dim_date.csv  
   - dim_hotels.csv
   - dim_rooms.csv
   - fact_aggregated_bookings
   - fact_bookings.csv

**Read bookings data in a datagrame**

In [2]:
df_bookings = pd.read_csv('datasets/fact_bookings.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/fact_bookings.csv'

**Explore bookings data**

In [ ]:
df_bookings.head()

In [ ]:
df_bookings.shape

In [ ]:
df_bookings.room_category.unique()

In [ ]:
df_bookings.booking_platform.unique()

In [ ]:
df_bookings.booking_platform.value_counts()

In [ ]:
df_bookings.booking_platform.value_counts().plot(kind="bar")

In [ ]:
df_bookings.describe()

**Read rest of the files**

In [ ]:
df_date = pd.read_csv('datasets/dim_date.csv')
df_hotels = pd.read_csv('datasets/dim_hotels.csv')
df_rooms = pd.read_csv('datasets/dim_rooms.csv')
df_agg_bookings = pd.read_csv('datasets/fact_aggregated_bookings.csv')

In [ ]:
df_hotels.shape

In [ ]:
df_hotels.head(3)

In [ ]:
df_hotels.category.value_counts()

In [ ]:
df_hotels.city.value_counts().plot(kind="bar")

***
**Exercise: Explore aggregate bookings**
***

In [ ]:
df_agg_bookings.head(3)

**Exercise-1. Find out unique property ids in aggregate bookings dataset**

In [ ]:
df_agg_bookings.property_id.unique()

**Exercise-2. Find out total bookings per property_id**

In [ ]:
df_agg_bookings.groupby("property_id")["successful_bookings"].sum()

**Exercise-3. Find out days on which bookings are greater than capacity**

In [ ]:
df_agg_bookings[df_agg_bookings.successful_bookings>df_agg_bookings.capacity]

**Exercise-4. Find out properties that have highest capacity**

In [ ]:
df_agg_bookings.capacity.max()

In [ ]:
df_agg_bookings[df_agg_bookings.capacity==df_agg_bookings.capacity.max()]

***
### ==> 2. Data Cleaning
***

In [ ]:
df_bookings.describe()

**(1) Clean invalid guests**

In [ ]:
df_bookings[df_bookings.no_guests<=0]

As you can see above, number of guests having less than zero value represents data error. We can ignore these records.

In [ ]:
df_bookings = df_bookings[df_bookings.no_guests>0]

In [ ]:
df_bookings.shape

**(2) Outlier removal in revenue generated**

In [ ]:
df_bookings.revenue_generated.min(), df_bookings.revenue_generated.max()

In [ ]:
df_bookings.revenue_generated.mean(), df_bookings.revenue_generated.median()

In [ ]:
avg, std = df_bookings.revenue_generated.mean(), df_bookings.revenue_generated.std()

In [ ]:
higher_limit = avg + 3*std
higher_limit

In [ ]:
lower_limit = avg - 3*std
lower_limit

In [ ]:
df_bookings[df_bookings.revenue_generated<=0]

In [ ]:
df_bookings[df_bookings.revenue_generated>higher_limit]

In [ ]:
df_bookings = df_bookings[df_bookings.revenue_generated<=higher_limit]
df_bookings.shape

In [ ]:
df_bookings.revenue_realized.describe()

In [ ]:
higher_limit = df_bookings.revenue_realized.mean() + 3*df_bookings.revenue_realized.std()
higher_limit

In [ ]:
df_bookings[df_bookings.revenue_realized>higher_limit]

One observation we can have in above dataframe is that all rooms are RT4 which means presidential suit. Now since RT4 is a luxurious room it is likely their rent will be higher. To make a fair analysis, we need to do data analysis only on RT4 room types

In [ ]:
df_bookings[df_bookings.room_category=="RT4"].revenue_realized.describe()

In [ ]:
# mean + 3*standard deviation
23439+3*9048

Here higher limit comes to be 50583 and in our dataframe above we can see that max value for revenue realized is 45220. Hence we can conclude that there is no outlier and we don't need to do any data cleaning on this particular column

In [ ]:
df_bookings[df_bookings.booking_id=="May012216558RT213"]

In [ ]:
df_bookings.isnull().sum()

Total values in our dataframe is 134576. Out of that 77899 rows has null rating. Since there are many rows with null rating, we should not filter these values. Also we should not replace this rating with a median or mean rating etc 

**Exercise-1. In aggregate bookings find columns that have null values. Fill these null values with whatever you think is the appropriate subtitute (possible ways is to use mean or median)**

In [ ]:
df_agg_bookings.isnull().sum()

In [ ]:
df_agg_bookings[df_agg_bookings.capacity.isna()]

In [ ]:
df_agg_bookings.capacity.median()

In [ ]:
df_agg_bookings.capacity.fillna(df_agg_bookings.capacity.median(), inplace=True)

In [ ]:
df_agg_bookings.loc[[8,15]]

**Exercise-2. In aggregate bookings find out records that have successful_bookings value greater than capacity. Filter those records**

In [ ]:
df_agg_bookings[df_agg_bookings.successful_bookings>df_agg_bookings.capacity]

In [ ]:
df_agg_bookings.shape

In [ ]:
df_agg_bookings = df_agg_bookings[df_agg_bookings.successful_bookings<=df_agg_bookings.capacity]
df_agg_bookings.shape

***
### ==> 3. Data Transformation
***

**Create occupancy percentage column**

In [ ]:
df_agg_bookings.head(3)

In [ ]:
df_agg_bookings['occ_pct'] = df_agg_bookings.apply(lambda row: row['successful_bookings']/row['capacity'], axis=1)

You can use following approach to get rid of SettingWithCopyWarning

In [ ]:
new_col = df_agg_bookings.apply(lambda row: row['successful_bookings']/row['capacity'], axis=1)
df_agg_bookings = df_agg_bookings.assign(occ_pct=new_col.values)
df_agg_bookings.head(3)

Convert it to a percentage value

In [ ]:
df_agg_bookings['occ_pct'] = df_agg_bookings['occ_pct'].apply(lambda x: round(x*100, 2))
df_agg_bookings.head(3)

In [ ]:
df_bookings.head()

In [ ]:
df_agg_bookings.info()

There are various types of data transformations that you may have to perform based on the need. Few examples of data transformations are,

1. Creating new columns
1. Normalization
1. Merging data
1. Aggregation

***
### ==> 4. Insights Generation
***

**1. What is an average occupancy rate in each of the room categories?**

In [ ]:
df_agg_bookings.head(3)

In [ ]:
df_agg_bookings.groupby("room_category")["occ_pct"].mean()

I don't understand RT1, RT2 etc. Print room categories such as Standard, Premium, Elite etc along with average occupancy percentage

In [ ]:
df = pd.merge(df_agg_bookings, df_rooms, left_on="room_category", right_on="room_id")
df.head(4)

In [ ]:
df.drop("room_id",axis=1, inplace=True)
df.head(4)

In [ ]:
df.groupby("room_class")["occ_pct"].mean()

In [ ]:
df[df.room_class=="Standard"].occ_pct.mean()

**2. Print average occupancy rate per city**

In [ ]:
df_hotels.head(3)

In [ ]:
df = pd.merge(df, df_hotels, on="property_id")
df.head(3)

In [ ]:
df.groupby("city")["occ_pct"].mean()

**3. When was the occupancy better? Weekday or Weekend?**

In [ ]:
df_date.head(3)

In [ ]:
df = pd.merge(df, df_date, left_on="check_in_date", right_on="date")
df.head(3)

In [ ]:
df.groupby("day_type")["occ_pct"].mean().round(2)

**4: In the month of June, what is the occupancy for different cities**

In [ ]:
df_june_22 = df[df["mmm yy"]=="Jun 22"]
df_june_22.head(4)

In [ ]:
df_june_22.groupby('city')['occ_pct'].mean().round(2).sort_values(ascending=False)

In [ ]:
df_june_22.groupby('city')['occ_pct'].mean().round(2).sort_values(ascending=False).plot(kind="bar")

**5: We got new data for the month of august. Append that to existing data**

In [ ]:
df_august = pd.read_csv("datasets/new_data_august.csv")
df_august.head(3)

In [ ]:
df_august.columns

In [ ]:
df.columns

In [ ]:
df_august.shape

In [ ]:
df.shape

In [ ]:
latest_df = pd.concat([df, df_august], ignore_index = True, axis = 0)
latest_df.tail(10)

In [ ]:
latest_df.shape

Check this post for codebasics resume project challange winner entry: https://www.linkedin.com/posts/ashishbabaria_codebasicsresumeprojectchallenge-data-powerbi-activity-6977940034414886914-dmoJ?utm_source=share&utm_medium=member_desktop

**6. Print revenue realized per city**

In [ ]:
df_bookings.head()

In [ ]:
df_hotels.head(3)

In [ ]:
df_bookings_all = pd.merge(df_bookings, df_hotels, on="property_id")
df_bookings_all.head(3)

In [ ]:
df_bookings_all.groupby("city")["revenue_realized"].sum()

**7. Print month by month revenue**

In [ ]:
df_date.head(3)

In [ ]:
df_date["mmm yy"].unique()

In [ ]:
df_bookings_all.head(3)

In [ ]:
df_date.info()

In [ ]:
df_date["date"] = pd.to_datetime(df_date["date"])
df_date.head(3)

In [ ]:
df_bookings_all.info()

In [ ]:
df_bookings_all["check_in_date"] = pd.to_datetime(df_bookings_all["check_in_date"])
df_bookings_all.head(4)

In [ ]:
df_bookings_all = pd.merge(df_bookings_all, df_date, left_on="check_in_date", right_on="date")
df_bookings_all.head(3)

In [ ]:
df_bookings_all.groupby("mmm yy")["revenue_realized"].sum()

**Exercise-1. Print revenue realized per hotel type**

In [ ]:
df_bookings_all.property_name.unique()

In [ ]:
df_bookings_all.groupby("property_name")["revenue_realized"].sum().round(2).sort_values()

**Exercise-2 Print average rating per city**

In [ ]:
df_bookings_all.groupby("city")["ratings_given"].mean().round(2)

**Exercise-3 Print a pie chart of revenue realized per booking platform**

In [ ]:
df_bookings_all.groupby("booking_platform")["revenue_realized"].sum().plot(kind="pie")

In [ ]:
import pandas as pd

In [ ]:
df_bookings = pd.read_csv(r'C:\Users\HP\Downloads\source-code\source-code\3_project_hospitality_analysis\datasets\fact_bookings.csv')
df_bookings.head(5)

In [ ]:
df_bookings.shape

In [ ]:
df_bookings.room_category.unique()

In [ ]:
df_bookings.booking_platform.unique()

In [ ]:
df_bookings.booking_platform.value_counts()

In [ ]:
df_bookings.booking_platform.value_counts().plot(kind='bar')

In [ ]:
df_bookings.describe()

In [ ]:
df_bookings.revenue_generated.min(), df_bookings.revenue_generated.max()

In [ ]:
df_date = pd.read_csv(r'C:\Users\HP\Downloads\source-code\source-code\3_project_hospitality_analysis\datasets\dim_date.csv')
df_hotels = pd.read_csv(r'C:\Users\HP\Downloads\source-code\source-code\3_project_hospitality_analysis\datasets\dim_hotels.csv')
df_rooms = pd.read_csv(r'C:\Users\HP\Downloads\source-code\source-code\3_project_hospitality_analysis\datasets\dim_rooms.csv')
df_agg_bookings = pd.read_csv(r'C:\Users\HP\Downloads\source-code\source-code\3_project_hospitality_analysis\datasets\fact_aggregated_bookings.csv')

In [ ]:
df_hotels.shape

In [ ]:
df_hotels.head(5)

In [ ]:
df_hotels.describe()

In [ ]:
df_hotels.category.value_counts()

In [ ]:
df_hotels.city.value_counts().plot(kind='bar')

In [ ]:
# Find out unique property ids om aggregate bookings dataset
# Find out total bookings per propertu_id
# Find out dayas on which bookings are greater than capacity
# Find out properties that have highest capacity

In [ ]:
# Find out unique property ids on aggregate bookings dataset

# df_agg_bookings.head(5)
df_agg_bookings.property_id.unique()


In [ ]:
# Find out total bookings per property_id

df_agg_bookings.groupby('property_id')["successful_bookings"].sum()

In [ ]:
# Find out days on which bookings are greater than capacity
df_agg_bookings[df_agg_bookings.successful_bookings > df_agg_bookings.capacity]

In [ ]:
df_agg_bookings.capacity.max()

In [ ]:
# Find out properties that have highest capacity
df_agg_bookings[df_agg_bookings.capacity==df_agg_bookings.capacity.max()]

## Data Cleaning

In [ ]:
df_bookings[df_bookings.no_guests <= 0]

In [ ]:
df_bookings[df_bookings.no_guests<0]

In [ ]:
df_bookings = df_bookings[df_bookings.no_guests>0]
df_bookings.shape

In [ ]:
df_bookings.revenue_generated.min(), df_bookings.revenue_generated.max()

In [ ]:
# removing outliers
df_bookings.revenue_generated.mean(), df_bookings.revenue_generated.std()

In [ ]:
avg, std = df_bookings.revenue_generated.mean(), df_bookings.revenue_generated.std()

In [ ]:
avg , std

In [ ]:
higher_limit = avg + 3*std
higher_limit

In [ ]:
lower_limit = avg - 3*std
lower_limit

In [ ]:
df_bookings[df_bookings.revenue_generated < 0]
df_bookings[df_bookings.revenue_generated > 0]

'''outlies seen on the higher limit in revenue_realized column'''

In [ ]:
df_bookings[df_bookings.revenue_generated > higher_limit]

In [ ]:
df_bookings = df_bookings[df_bookings.revenue_generated < higher_limit]
df_bookings.shape

In [ ]:
df_bookings.revenue_generated.describe()

In [ ]:
higher_limit = df_bookings.revenue_generated.mean() + 3*df_bookings.revenue_realized.std()
higher_limit

In [ ]:
df_bookings[df_bookings.revenue_realized>higher_limit]

In [ ]:
df_rooms

In [ ]:
df_bookings[df_bookings.room_category == 'RT4'].revenue_realized.describe()

In [ ]:
23439 + 3*9048 
# there is no outiliers in revenue realized

In [ ]:
df_bookings.isnull().sum()

## Data Transformation

In [ ]:
df_agg_bookings.head()

In [ ]:
25/30

In [ ]:
df_agg_bookings['occt_pct'] = df_agg_bookings['successful_bookings']/df_agg_bookings['capacity']
df_agg_bookings.head()

In [ ]:
# applying lambda function to round the occupancy % of the table to 2

df_agg_bookings['occt_pct'] = df_agg_bookings['occt_pct'].apply(lambda x: round(x*100, 2))
df_agg_bookings.head(4)

In [ ]:
# applying lambda to get difference of perentage

df_agg_bookings['occt_diff'] = df_agg_bookings.apply(lambda x : x['capacity']-x['successful_bookings'], axis =1)

In [ ]:
df_agg_bookings.head(5)

In [ ]:
# applying la,bda function to counter rise in capacity above threshold limit
# maintainance alert

''''
condition = df_agg_bookings['occt_pct'] > 100 
def set_alert(row):
    if row['occt_pct'] > 100.00:
        return 'Y'
    else:
        return 'N'
df_agg_bookings['Alert'] = df_agg_bookings.apply(set_alert, axis = 1)
'''
df_agg_bookings['Alert'] = df_agg_bookings.apply(lambda row: '1' if row['occt_pct'] > 100 else '0', axis=1)

In [ ]:
df_agg_bookings.head()

## NORMALIZATION

## MERGE

## Aggregation

1. what is the average occipancy rate in each of the room catogries?

In [ ]:
df_agg_bookings.groupby('room_category')['occt_pct']. mean().round(2)


In [ ]:
df_rooms

In [ ]:
df = pd.merge(df_agg_bookings, df_rooms, left_on = 'room_category', right_on = 'room_id')


In [ ]:
df.tail(4)

In [ ]:
df.groupby('room_class')['occt_pct'].mean().round(2)

In [ ]:
df.drop('room_id', axis=1, inplace = True)  #inplace: modify that dataframe, axis=1 is Column
df.head(4)

2.Printing avg occupancy rate per city

In [ ]:
df_hotels.head(3)

In [ ]:
df = pd.merge(df, df_hotels, on = 'property_id', how='left')
df.head(3)

In [ ]:
df.groupby('city')['occt_pct'].mean().plot(kind = 'bar')

3 when was the occupancy better? weekday or weekend?

In [ ]:
df_date.head(3)

In [ ]:
df = pd.merge(df, df_date, left_on='check_in_date', right_on='date')
df.head(3)

In [ ]:
df.groupby('day_type')['occt_pct'].mean().round(2)

4. In the month of june what is the occupancy for different cities

In [ ]:
df['mmm yy'].unique()

In [ ]:
df_june_22 = df[df['mmm yy']=='Jun 22']
df_june_22.head(4)

In [ ]:
df_june_22.groupby('city')['occt_pct'].mean().round(2).sort_values(ascending=False)

In [ ]:
df_august = pd.read_csv(r'C:\Users\HP\Downloads\source-code\source-code\3_project_hospitality_analysis\datasets\new_data_august.csv')
df_august.head(4)

In [ ]:
df_august.columns

In [ ]:
df.columns

In [ ]:
df_august.shape

In [ ]:
df.shape

In [ ]:
latest_df = pd.concat([df,df_august], ignore_index=True, axis=0)
latest_df.tail(5)

In [ ]:
latest_df.shape

6. Revenue realized per city

In [ ]:
df_bookings.head()

In [ ]:
df_hotels.head(4)

In [ ]:
df_bookings_all = pd.merge(df_bookings, df_hotels, on='property_id')
df_bookings_all.head(3)

In [ ]:
df_bookings_all.groupby('city')['revenue_realized'].sum()

## 7. Print month by month

In [ ]:
df_date.head(3)

In [ ]:
df_date['mmm yy'].unique()


In [ ]:
df_bookings_all.head(3)

In [ ]:
df_date.info()

In [ ]:
# converting date from obj to date type
df_bookings_all["check_in_date"] = pd.to_datetime(df_bookings_all["check_in_date"])
df_bookings_all.head(4)

In [ ]:
# converting date from obj to date type
df_bookings_all['check_in_date']= pd.to_datetime(df_date['date'])
df_bookings_all.head(3)

In [ ]:
df_bookings_all.groupby("mmm yy")["revenue_realized"].sum()

## Revenue realized per hoteltype

In [ ]:
df_bookings_all.property_name.unique()

In [ ]:
df_bookings_all.groupby("city")["ratings_given"].mean().round(2)